In [1]:
#New compilation of previous functions to include activation markers under predictPopulations() /Tuesday morning 18Jun

In [1]:
import pandas
import numpy
from sklearn import mixture
from sklearn.mixture import GaussianMixture
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
fllst=['20161028_KW_24_Control_NORM.csv', '20161027_KW_31_CONTROL_NORM.csv', '20161027_KW_30_CONTROL_NORM.csv','20161028_KW_1_Control_NORM.csv', '20161027_KW_51_CONTROL_NORM.csv', '20161027_KW_43_CONTROL_NORM.csv', '20161026_18_Control_NORM.csv','20161026_17_Control_NORM.csv', '20170216 KW 14Con_NORM.csv', '20170216 KW 22 Con_NORM.csv','20170316 KW 41 Con_NORM.csv','20170316 KW 42 Con_NORM.csv', '20170316 KW 4 Con_NORM.csv', '20170329 KW 80con_NORM.csv', '20170216 KW 63 Con_NORM.csv',  '20161028_KW_3_Control_NORM.csv','20170317KW JC Con.csv','20170317KW KW Con.csv','20170317KW IH Con.csv','20161028_KW_1_FLAG_NORM.csv', '20161028_KW_3_FLAG_NORM.csv', '20170316 KW 4 Flag_NORM.csv', '20170216 KW 14 FLAG_NORM.csv', '20161026_17_FLAG_NORM.csv', '20161026_18_FLAG_NORM.csv', '20170216 KW 22 Flag_NORM.csv', '20161028_KW_24_FLAG_NORM.csv', '20161027_KW_30_FLAG_NORM.csv', '20161027_KW_31_FLAG_NORM.csv', '20170316 KW 41 Flag_NORM.csv', '20170316 KW 42 Flag_NORM.csv', '20161026_43_FLAG_NORM.csv', '20161027_KW_51_FLAG_NORM.csv', '20170216 KW 63 Flag_NORM.csv', '20170329 KW 80 Flag_NORM.csv', '20170317KW JC Flag.csv', '20170317KW KW Flag.csv', '20170317KW IH Flag.csv','20161028_KW_1_IFN_NORM.csv', '20161028_KW_3_IFN_NORM.csv', '20170316 KW 4 INF_NORM.csv', '20170216 KW 14 IFN_NORM.csv', '20161026_17_IFN_NORM.csv', '20161026_18_IFN_NORM.csv', '20170216 KW 22 IFN_NORM.csv', '20161028_KW_24_IFN_NORM.csv', '20161027_KW_30_IFN_NORM.csv', '20161027_KW_31_IFN_NORM.csv', '20170316 KW 41 INF_NORM.csv', '20170316 KW 42 INF_NORM.csv', '20161027_KW_51_IFN_NORM.csv', '20170216 KW 63 IFN_NORM.csv', '20170329 KW 80 IFN_NORM.csv', '20170317KW JC INF.csv', '20170317KW KW IFN.csv', '20170317KW IH INF.csv']


In [3]:
#Dictionaries that hold the detail of the Type antigens (typmrks) & the activation antigens (actmrks);
#the keys are the column names in the files (based on metal isoptope) & the values are the common names of the antigens


typmrks = {
    "Y89Di":"CD3",
    "Bi209Di":"CD4",
    "Nd142Di":"CD19",
    "Er168Di":"CD8",
    "Eu151Di":"CD14",
  
}

actmrks = {
    "Nd143Di":"HLA-DR",
    "Sm147Di":"CD86",
    "Sm149Di":"CD25",
    "Nd150Di":"pSTAT5",
    "Sm152Di":"pSTAT1",
    "Sm154Di":"NKp30",
    "Gd155Di":"pMAPKAPK2",
    "Gd156Di":"pP38",
    "Gd158Di":"pSTAT3",
    "Tb159Di":"CD161",
    "Gd160Di":"T-Bet",
    "Dy161Di":"TLR4",
    "Dy162Di":"FoxP3",
    "Dy164Di":"Arginase-1",
    "Ho165Di":"pCREB",
    "Er166Di":"NFkB p65",
    "Er170Di":"TCR Vα24-Ja18",
    "Yb171Di":"pERK 1/2",
    "Yb174Di":"pSTAT2",
    "Lu175Di":"TLR5",
    "Yb176Di":"TLR2"

}

In [4]:
#Function that filters ("gates") the data (rows) - in this case hard coded for Event_length < 4.5 & (DNA1>7.5 or DNA2>8)
#Receives the files & type marker dictionary & 
#Returns a dataframe (1) containing the antigens defined as TypeMarkers (typmrks) in the relevant dictionary for the gated 
#rows & (2) all the antigens for the rows gated according to the criteria

def gateDNA_Event_length(fle,typmrks):
    df=pandas.read_csv(fle,delimiter=',')
    filtereddf = df[(numpy.log(df["Event_length"]+1)<4.5) & ((numpy.log(df["Ir191Di"]+1)>7.5) | (numpy.log(df["Ir193Di"]+1)>8.0))]
    data = {}
    for k in typmrks.keys():
        data[typmrks[k]] = filtereddf[k]
    dtafrm=pandas.DataFrame(data)
    dtafrm2=pandas.DataFrame(filtereddf)
    return (dtafrm,dtafrm2)


In [5]:
#Function that predicts the populations using Gaussian Mixture Models where the population number is specified.
#The predictions include populations (means for the axes), population sizes & the means of the activation makers for the
#predicted populations
#It receives in order 1) the gated dataframe for the antigens for which the popualtions is to be predicted (dt), the number
#of specified populatuions (ngroups), the activation markers for which the means are needed (actmrks) and the gated dataframe
#that contains those activation markers
#It returns 1) the predicted means of the axes for the antigens, 2) the sizes of the predicted populations & 3) the means of
#the defined activation markers for the antigens


def predictPopulations(dt,ngroups,actmrks,dtafrm2):
    Classifier = dt.sample(frac=0.25)    
    model=GaussianMixture(n_components=ngroups,tol=0.001,n_init=5).fit(numpy.log(Classifier+1))
    predictions = model.predict(numpy.log(dt+1))
    counts = []
    Actmrkrs=[]
    
    for p in range(ngroups):
        counts.append(sum(predictions==p))
        actvtn=[]
        for a in actmrks.keys():
            actvtn.append((numpy.mean(numpy.log(dtafrm2+1).loc[predictions==p,a])))
        Actmrkrs.append(actvtn)
    
    means = pandas.DataFrame(model.means_)
    means.columns = Classifier.columns # assign column names
    actvtnMarkers=pandas.DataFrame(Actmrkrs)
    actvtnMarkers.columns=(actmrks.values())
    actvtnMarkers["PopSize"]=counts
    return (means,counts,actvtnMarkers)


In [6]:
#Function that selects the highest & second highest mean value for a specific antigen.
#It receives as arguments the antigen name, the file number and the number of specified populations, and
#Returns the ROW number of the highest & second highest means

def largestNumber(x,i,ngroups):
    '''The column name & file number is passed and the number of the highest row is returned'''
        
    CDnum1=0
    CDnum2=0
    CDnum3=0
    largestCD=0
    scndLargest=0
    thrdLargest=0
    
    for j in range (ngroups):
        CD=axesMeans[i][x][j]
        if CD>largestCD:
            scndLargest=largestCD
            largestCD=CD
            CDnum2=CDnum1
            CDnum1=j
        elif CD>scndLargest:
            scndLargest=CD
            CDnum2=j
        elif CD>thrdLargest:
            thrdLargest=CD
            CDnum3=j

            
    return (CDnum1,CDnum2,CDnum3)

In [7]:
#Function to return population size when population "number" is entered (row number)
def popSize(y,x):
    '''Population number & file number is passed & population size is returned'''
    if y == 0:
        pop=totPop1[x]
    elif y== 1:
        pop=totPop2[x]
    elif y == 2:
        pop=totPop3[x]
    elif y == 3:
        pop=totPop4[x]
    elif y == 4:
        pop=totPop5[x]
    return pop

In [8]:
def checkCD14(cd4,cd8,cd19):
    cd14=popSize((largestNumber('CD14',i,ngroups)[0]),i)
    if cd14==cd4 or cd14==cd8 or cd14==cd19:
        cd14=cd14=popSize((largestNumber('CD14',i,ngroups)[1]),i)
        if cd14==cd4 or cd14==cd8 or cd14==cd19:
            cd14=cd14=popSize((largestNumber('CD14',i,ngroups)[2]),i)
    return cd14

In [9]:
#Code that run the functions

for z in range (0,7):
    print z+1
    n=len(fllst)

    axesMeans=[]
    totPop1=[]
    totPop2=[]
    totPop3=[]
    totPop4=[]
    totPop5=[]

    fleName=[]
    totFile=[]
    flelen=[]

    Cd3=[]
    Cd4=[]
    Cd8=[]
    Cd19=[]
    Cd14=[]
    Cd3per=[]

    Cd4act=[]
    Cd8act=[]
    Cd14act=[]
    Cd19act=[]


    ngroups=5

    for i in range (0,n):
        fle=fllst[i]
        #print i+1,fle # Keeps track of files as processed

        dt = gateDNA_Event_length(fle,typmrks)
        dataset=predictPopulations(dt[0],ngroups,actmrks,dt[1])

        flelength=dt[0].shape[0]

        axesMeans.append(dataset[0])
        totPop1.append(dataset[1][0])
        totPop2.append(dataset[1][1])
        totPop3.append(dataset[1][2])
        totPop4.append(dataset[1][3])
        totPop5.append(dataset[1][4])
        fleName.append(fle)
        totFile.append(dataset[1][0]+dataset[1][1]+dataset[1][2]+dataset[1][3]+dataset[1][4])
        flelen.append(flelength)

        cd4=popSize((largestNumber('CD4',i,ngroups)[0]),i)
        cd8=popSize((largestNumber('CD8',i,ngroups)[0]),i)
        cd19=popSize((largestNumber('CD19',i,ngroups)[0]),i)
        #cd14=popSize((largestNumber('CD14',i,ngroups)[1]),i)
        cd14=checkCD14(cd4,cd8,cd19)

        cd3 = cd4+cd8
        cd3per = int(float(cd3)/float(totFile[i])*100)

        Cd4.append(cd4)
        Cd8.append(cd8)
        Cd19.append(cd19)
        Cd3.append(cd3)
        Cd3per.append(cd3per)
        Cd14.append(cd14)

        cd4act=(dataset[2].loc[dataset[2]['PopSize']==cd4])
        cd4actmr=[]
        for j in range (0,len(actmrks)):
            cd4actmr.append(cd4act.iloc[0][j])
        Cd4act.append(cd4actmr)

        cd8act=(dataset[2].loc[dataset[2]['PopSize']==cd8])
        cd8actmr=[]
        for j in range (0,len(actmrks)):
            cd8actmr.append(cd8act.iloc[0][j])
        Cd8act.append(cd8actmr)

        cd14act=(dataset[2].loc[dataset[2]['PopSize']==cd14])
        cd14actmr=[]
        for j in range (0,len(actmrks)):
            cd14actmr.append(cd14act.iloc[0][j])
        Cd14act.append(cd14actmr)

        cd19act=(dataset[2].loc[dataset[2]['PopSize']==cd19])
        cd19actmr=[]
        for j in range (0,len(actmrks)):
            cd19actmr.append(cd19act.iloc[0][j])
        Cd19act.append(cd19actmr)


    df2=pandas.DataFrame({'FileName':fleName,
                          'AxesMeans':axesMeans, 
                          'TotalPopulation1':totPop1,
                          'TotalPopulation2':totPop2,
                          'TotalPopulation3':totPop3, 
                          'TotalPopulation4':totPop4,
                          'TotalPopulation5':totPop5,
                          'CD3':Cd3,
                          'CD4':Cd4,
                          'CD8':Cd8,
                          'CD14':Cd14,
                          'CD19':Cd19, 
                          'DatasetTotal':totFile,
                          'DatasetTotal2':flelen,
                          'CD3%':Cd3per})

    df3=pandas.DataFrame({'FileName':fleName,
                          'CD3':Cd3 ,
                          'CD4':Cd4,
                          'CD8':Cd8,
                          'CD19':Cd19,
                          'CD14':Cd14,
                          'DatasetTotal':totFile})

    df4=pandas.DataFrame(Cd4act)
    df4.columns=(actmrks.values())
    df4['FileName']=fleName

    df8=pandas.DataFrame(Cd8act)
    df8.columns=(actmrks.values())
    df8['FileName']=fleName

    df14=pandas.DataFrame(Cd14act)
    df14.columns=(actmrks.values())
    df14['FileName']=fleName

    df19=pandas.DataFrame(Cd19act)
    df19.columns=(actmrks.values())
    df19['FileName']=fleName

    name='File'+str(z+1)+'.csv'
    name2='ResultFile_allDetail#5_'+str(z+1)+'.xls'
    name3='ResultFile_allDetail#5_'+str(z+1)+'.csv'
    name4='ActivationMarkers_CD4_'+str(z+1)+'.csv'
    name8='ActivationMarkers_CD8_'+str(z+1)+'.csv'
    name14='ActivationMarkers_CD14_'+str(z+1)+'.csv'
    name19='ActivationMarkers_CD19_'+str(z+1)+'.csv'

    df2.to_excel(name2)
    df3.to_csv(name3)
    df4.to_csv(name4)
    df8.to_csv(name8)
    df14.to_csv(name14)
    df19.to_csv(name19)

print "Done!"


1
2
3
4
5
6
7
Done!
